In [1]:
import numpy as np
import pandas
import geopandas
from shapely.geometry import Point
from IPython.display import display, clear_output
import pptk
from sklearn.neighbors import KDTree
import time
import random
random.seed(1337)

In [2]:
def pointRound(x, base=5):
    return base * round(x/base)

In [3]:
def cluster(point_array, base=5):
    tmp = 0
    counter = 0

    lidar_points_clustered = np.copy(lidar_points)

    length = lidar_points_clustered.shape[0]

    for i in range(0, length):

        if(tmp == 9000):
            tmp = 0
            clear_output(wait=True)
            display("progress: " + str(counter / length))

        counter += 1
        tmp += 1

        lidar_points_clustered[i][0] = pointRound(lidar_points_clustered[i][0], base)
        lidar_points_clustered[i][1] = pointRound(lidar_points_clustered[i][1], base)
        lidar_points_clustered[i][2] = pointRound(lidar_points_clustered[i][2], base)

    #lidar_points = np.unique(lidar_points)

    print("before, there were: " + str(length) + " points")

    unique = np.unique(lidar_points_clustered, axis=0)
    
    print("afterwards, there were: " + str(unique.shape[0]) + " points")
    
    return unique

In [4]:
def isInBox(point_x, point_y , bounds):
    #bounds = (minx, miny, maxx, maxy)
    return ((point_x > bounds[0]) and (point_x < bounds[2]) and (point_y > bounds[1]) and (point_y < bounds[3]))

In [5]:
def crossProdTest(point_a, point_b, point_c):
    if ((point_a[1] == point_b[1]) and (point_b[1] == point_c[1])):
        if ((point_b[0] <= point_a[0]) and (point_a[0] <= point_c[0]) or (point_c[0] <= point_a[0]) and (point_a[0] <= point_b[0])):
            return 0
        else:
            return 1
        
    if ((point_a[1] == point_b[1]) and (point_a[0] == point_b[0])):
        return 0
    
    if (point_b[1] > point_c[1]):
        point_b, point_c = point_c, point_b
    
    if ((point_a[1] <= point_b[1]) or (point_a[1] > point_c[1])):
        return 1
    
    delta = (point_b[0] - point_a[0]) * (point_c[1] - point_a[1]) - (point_b[1] - point_a[1]) * (point_c[0] - point_a[0])
    
    if (delta > 0):
        return -1
    elif (delta < 0):
        return 1
    else:
        return 0

In [6]:
def isPointInPolygon(point, polygon):
    #polygon = np.insert(polygon, 0, polygon[-1])    
    polygon = [polygon[-1]] + polygon
    t = -1
    
    for i in range(0, (len(polygon) - 1)):
        t = t * crossProdTest(point, polygon[i], polygon[i + 1])
        if (t == 0):
            return True
    return (t >= 0)
    

In [7]:
def random_points_within(poly, bounds, stepping_size = 0.1, z_coord):
    min_x, min_y, max_x, max_y = bounds  
    
    points = []
    
    x_steps = (max_x - min_x) / stepping_size
    
    y_steps = (max_y - min_y) / stepping_size
    
    for i in  range(0, x_steps):
        for j in range(0, y_steps):
            next_point = [min_x + (i * stepping_size), min_y + (j * stepping_size), z_coord]
            
            if (isPointInPolygon(next_point, poly)):
                points.append(next_point)
                
    return points

In [ ]:
def get_nearest_transport(point):
   #point = (x , y)
   max_distance = 100
   test = []
   for transport in transport_shape['geometry']:
       test.append([transport.x, transport.y])

   transport_tree = KDTree(test)
   nearst_tranport = transport_tree.query(np.array([point[0], point[1]]).reshape(1,-1), 1)
   return transport_shape.values[nearst_tranport[1][0][0]]

In [8]:
shape_file = "data/colonge_small/shp_koeln/gis_osm_buildings.shp"
shape  = geopandas.read_file(shape_file)

transport_shape = "./data/colonge_small/shp_koeln/gis_osm_transport.shp"
transport_shape = geopandas.read_file(transport_shape)
transport_shape.drop( transport_shape[ transport_shape['fclass'] == 'taxi' ].index , inplace=True)

lidar_file = "data/colonge_small/dom1l-fp_32356_5645_1_nw.xyz"
#read the file into a numpy array
lidar_points = np.loadtxt(open(lidar_file, "r"), delimiter=",", skiprows=0)

In [9]:
max_index = shape.index.max()

list_polygons = []

for i in range(0, (max_index + 1)):
    current_object = shape.loc[i]
    
    shape_name = current_object['name']
    shape_type = current_object['type']
    
    if (shape_name == None and (shape_type == None or shape_type == "appartment" or shape_type == "house")):
        #print(current_object['geometry'])
        geometry_object = current_object['geometry']
        
        #print(geometry_object.size)

        list_polygons.append([geometry_object.exterior.coords[:], geometry_object.bounds, 0, True])
        
        pure = geometry_object.exterior.coords[:]

In [15]:
clustered = cluster(lidar_points, 1)
#lidar_clustered_z_sorted_desc = np.flip(clustered[clustered[:,2].argsort()], 0)
lidar_clustered_z_sorted_desc = np.flip(lidar_points[lidar_points[:,2].argsort()], 0)

'progress: 0.9999635714545918'

before, there were: 8235300 points
afterwards, there were: 2097972 points


In [11]:
tree = KDTree(lidar_clustered_z_sorted_desc)

In [12]:
max_distance = 50
standart_height = 80

length = len(list_polygons)

counter = 0

not_found = 0

for polygon in list_polygons:
    
    clear_output(wait=True)
    display("progress: " + str(counter / length))
    counter += 1
    
    #bounds = (minx, miny, maxx, maxy)
    mid_x = (polygon[1][0] + polygon[1][2]) / 2 #find the mid point for the x coordinate
    mid_y = (polygon[1][1] + polygon[1][3]) / 2 #find the mid point for the y coordinate
    nearest_neighbors = (tree.query_radius(np.array([mid_x, mid_y, standart_height]).reshape(1, -1), max_distance))
    
    nearest_neighbors = np.sort(nearest_neighbors[0])
    
    for neighbor_index in nearest_neighbors:
        neighbor_point = lidar_clustered_z_sorted_desc[neighbor_index]
        
        if(polygon[3]):
            if (isInBox(neighbor_point[0], neighbor_point[1], polygon[1])):
                if(isPointInPolygon(neighbor_point, polygon[0])):
                    polygon[2] = neighbor_point[2]
                    polygon[3] = False
                    print(".")
                    break
    
    if(polygon[3]):
        #print("exhausted all " + str(len(nearest_neighbors)) + " but found not fitting one")
        not_found += 1

print("not found: " + str(not_found) + " of " + str(length))

'progress: 0.9995226730310263'

.
not found: 950 of 2095


In [34]:
red = np.array([1,0,0])

new_points = lidar_points

print("davor: " + str(lidar_points.shape[0]))

current_index = 0

counter = 0

not_found = 0

for polygon in list_polygons:
    
    clear_output(wait=True)
    display("progress: " + str(counter / length))
    counter += 1
    if(not(polygon[3])): #a value was set
        
        rand_points = np.array(random_points_within(polygon, polygon[1], 0.1, polygon[2])[0]).reshape(1,3)
        
        new_points = np.append(new_points, rand_points, axis=0)
        
total_size = new_points.shape[0]

new_color = np.full((total_size, 3), 1)

for i in range(lidar_points.shape[0], total_size):
    new_color[i] = red

print("new_c: " + str(new_color.shape[0]))
 
print("new_p: " + str(new_points.shape[0]))


v = pptk.viewer(new_points, new_color)

'progress: 0.9995226730310263'

AttributeError: 'tuple' object has no attribute 'shape'

In [30]:
print(new_color.shape[0])
print(new_points.shape[0])

8236445
8236445
